<a href="https://colab.research.google.com/github/priyadarshinikp1/Genomics/blob/main/genomics_mt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We Can Call mtDNA + Nuclear DNA Together using GATK HaplotypeCaller

This will call autosomes + sex chromosomes + mtDNA, but...

Problem: mtDNA is haploid, not diploid. This can lead to:

Incorrect genotypes (e.g. 0/1 or 1/1 instead of 1)

Misleading allele depths and likelihoods

🛠 Best Practice (Recommended Hybrid Approach):

✅ Step 1: Standard Nuclear Variant Calling

Call nuclear variants with default diploid settings (ploidy=2), excluding mitochondrial contig.
Create the autosomes.sex.chroms.list by excluding "MT" or "chrM".

In [ ]:
gatk HaplotypeCaller \
  -R reference.fasta \
  -I sample.bam \
  -O sample.nuclear.vcf.gz \
  -L autosomes.sex.chroms.list \
  --sample-ploidy 2

✅ Step 2: mtDNA-Specific Variant Calling

Call only mtDNA, with --sample-ploidy 1:

In [ ]:
gatk HaplotypeCaller \
  -R reference.fasta \
  -I sample.bam \
  -O sample.mtdna.vcf.gz \
  -L MT \
  --sample-ploidy 1

✅ Step 3: Merge VCFs (Optional)

If you want to combine nuclear and mtDNA VCFs:

In [ ]:
bcftools concat -a -Oz -o sample.all.vcf.gz sample.nuclear.vcf.gz sample.mtdna.vcf.gz
bcftools index sample.all.vcf.gz

✅ Step 4: Annotation (e.g., with VEP or ANNOVAR)

Now annotate the merged VCF, or annotate nuclear and mtDNA VCFs separately.

In [ ]:
vep -i sample.all.vcf.gz -o sample.annotated.vcf --vcf --everything --assembly GRCh38

annotation can be done using web based tools like mitomap or mitomaster